## 【ベイズ決定】肉まんの販売個数

モデルから予測された事後予測分布を使って、蒸し器に投入する最適な肉まんの個数を予測させる。

In [ ]:
import pymc as pm
import arviz as az

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = [10, 4]

## Load & Preprocess Data

In [ ]:
data = pd.read_csv('data.csv')

In [ ]:
x = data['temperature'].values
y = data['num_sold'].values

store_id = data['store_id'].values

num_stores = np.max(store_id) + 1

In [ ]:
x_mu = np.mean(x)
x_sd = np.std(x)

x_scaled = (x - x_mu) / x_sd

## Define & Load Model

In [ ]:
with pm.Model() as model:

    shared_x_scaled = pm.Data('x_scaled', x_scaled, mutable=True)
    shared_store_id = pm.Data('store_id', store_id, mutable=True)

    shared_y = pm.Data('y', y, mutable=True)

    a = pm.Normal('a', mu=0, sigma=10)
    b = pm.Normal('b', mu=0, sigma=10)

    s = pm.HalfCauchy('s', 5)
    r = pm.Normal('r', mu=0, sigma=s, shape=(num_stores,))

    theta = a * shared_x_scaled + r[shared_store_id] + b

    mu = pm.math.exp(theta)

    obs = pm.Poisson('obs', mu=mu, observed=shared_y)

In [ ]:
idata = az.from_netcdf('idata.nc')

## Bayes Decision

気温が５度の日に、ある店舗（ID=6）で何個の肉まんを蒸し器にいれるべきかを考える。

In [ ]:
x_new = [5]

x_scaled_new = (x_new - x_mu) / x_sd

In [ ]:
store_id_new = [6]

In [ ]:
y_dummy = np.zeros_like(x_new, dtype=int)

In [ ]:
with model:

    pm.set_data({
        'x_scaled': x_scaled_new,
        'store_id': store_id_new,
        'y': y_dummy
    })

    ppc = pm.sample_posterior_predictive(idata.posterior, return_inferencedata=False, var_names=['obs'])

In [ ]:
num_sales_pred = np.array(ppc['obs'])
num_sales_pred

In [ ]:
num_sales_pred.shape

In [ ]:
az.plot_dist(num_sales_pred)

plt.xlabel('Number of Sales')
plt.ylabel('Probability');

In [ ]:
def estimate_profit(num_sales_pred, num_steamed):

    profit = np.minimum(num_sales_pred, num_steamed) * 150 - 50 * num_steamed

    return np.mean(profit)

In [ ]:
num_porkbuns_to_steam = np.arange(10)

profit_estimated = [estimate_profit(num_sales_pred, k) for k in num_porkbuns_to_steam]

for k in num_porkbuns_to_steam:

    print('肉まんの個数: {}, 利益: {:3.0f}円'.format(k, np.round(profit_estimated[k])))

In [ ]:
plt.plot(profit_estimated, 'o-')
plt.xticks(np.arange(len(profit_estimated)))

plt.xlabel('Number to Steam')
plt.ylabel('Expected Profit');

## Combine Weather Prediction

天気予報では気温が５度、予測精度が標準偏差１であるとした場合に、蒸し器に入れるべき肉まんの個数を考える。

In [ ]:
x_new_mu = 5
x_new_sd = 1

In [ ]:
x_new = np.random.normal(loc=x_new_mu, scale=x_new_sd, size=1000)

In [ ]:
fig = plt.subplots(figsize=(10, 4))

sns.histplot(x_new)
plt.xlabel('Temperature');

In [ ]:
x_scaled_new = (x_new - x_mu) / x_sd

store_id_dup = np.ones_like(x_new, dtype=int) * store_id_new

y_dummy = np.zeros_like(x_new, dtype=int)

In [ ]:
with model:

    pm.set_data({
        'x_scaled': x_scaled_new,
        'store_id': store_id_dup,
        'y': y_dummy
    })

    ppc = pm.sample_posterior_predictive(idata.posterior, return_inferencedata=False, var_names=['obs'])

In [ ]:
num_sales_pred = np.array(ppc['obs'])
num_sales_pred

In [ ]:
num_sales_pred.shape

In [ ]:
az.plot_dist(num_sales_pred)

plt.xlabel('Number of Sales')
plt.ylabel('Probability');

In [ ]:
num_porkbuns_to_steam = np.arange(10)

profit_estimated = [estimate_profit(num_sales_pred, k) for k in num_porkbuns_to_steam]

for k in num_porkbuns_to_steam:

    print('肉まんの個数: {}, 利益: {:3.0f}円'.format(k, np.round(profit_estimated[k])))

In [ ]:
plt.plot(profit_estimated, 'o-')
plt.xticks(np.arange(len(profit_estimated)))

plt.xlabel('Number to Steam')
plt.ylabel('Expected Profit');